In [1]:
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift

In [2]:
def updownlimit_old(df_nowprice_pct,df_weight):
    '''
    INPUT:
    df_nowprice_pct:为当前时间点的价格相较于昨日收盘价的涨跌幅，如现在为10:30，则为10:30_price - precloseprice
    df_weight：当日交易的目标权重，如今日想要半仓买入某只股票，则当日的weight为0.5，做空反之
    OUTPUT:
    df_weight_final 考虑了当前时间点由于涨跌停限制买不进去情况下的最终权重，其中不能换仓的资产延续上一个交易日的权重
    '''
    # TODO 写成 numpy
    # TODO 类型检验
    if df_nowprice_pct.shape != df_weight.shape:
        raise TypeError("input data must has same dims.")
    df_islimit = (~((abs(df_nowprice_pct) > 0.097))).astype("float32").replace(0.,np.nan) # 当天涨跌停为nan，其它为1
    df_isupdown = ((abs(df_nowprice_pct) > 0.097)).astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan
    df_isupdown_weight = df_isupdown.shift(-1)*df_weight              # 涨跌停前一天的权值

    df_weight_outlimit = (df_islimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值

    df_weight_limitday = ((df_isupdown_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_islimit*df_weight).fillna(0)# 涨跌停当天的权值,按照前一个交易日的权重进行填充

    sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
    df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
    df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重

    df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)
    return df_weight_final

In [3]:
def updownlimit(df_nowprice_pct,df_weight):
    '''
    INPUT:
    df_nowprice_pct:为当前时间点的价格相较于昨日收盘价的涨跌幅，如现在为10:30，则为10:30_price - precloseprice
    df_weight：当日交易的目标权重，如今日想要半仓买入某只股票，则当日的weight为0.5，做空反之
    OUTPUT:
    df_weight_final 考虑了当前时间点由于涨跌停限制买不进去情况下的最终权重，其中不能换仓的资产延续上一个交易日的权重
    '''
    # TODO 写成 numpy
    # TODO 类型检验
    df_islimit = (~((abs(df_nowprice_pct) > 0.097) ) ).astype("float32").replace(0.,np.nan) # 当天涨跌停为nan，其它为1
    df_isupdown = df_islimit.isnull().astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan

    df_limitbefore_weight = (df_isupdown.shift(-1)*df_weight.fillna(0))            # 涨跌停前一天的权值
    df_weight_outlimit = (df_islimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值
    df_weight_limitday = ((df_limitbefore_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_limitbefore_weight*df_islimit).fillna(0) # 涨跌停当天的权值,按照前一个交易日的权重进行填充
    sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
    df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
    df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重
    df_weight_final_temp = df_weight_outlimit + df_weight_limitday.fillna(0)

    # 再来一遍
    df_limitbefore_weight = (df_isupdown.shift(-1)*df_weight_final_temp.fillna(0))            # 涨跌停前一天的权值
    df_weight_outlimit = (df_islimit*df_weight_final_temp).fillna(0) 
    df_weight_limitday = ((df_limitbefore_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_limitbefore_weight*df_islimit).fillna(0) # 涨跌停当天的权值,按照前一个交易日的权重进行填充
    sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
    df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
    df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重
    df_weight_final_temp = df_weight_outlimit + df_weight_limitday.fillna(0)
    
    # 再来一遍
    df_islimit = (~df_nowprice_pct.isnull()).astype("float32").replace(0.,np.nan) #  
    df_isupdown = df_islimit.isnull().astype("float32").replace(0.,np.nan) #  
    df_limitbefore_weight = (df_isupdown.shift(-1)*df_weight_final_temp.fillna(0))            # 涨跌停前一天的权值
    df_weight_outlimit = (df_islimit*df_weight_final_temp).fillna(0) 
    df_weight_limitday = ((df_limitbefore_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_limitbefore_weight*df_islimit).fillna(0) # 涨跌停当天的权值,按照前一个交易日的权重进行填充
    sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
    df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
    df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重
    df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)
    
    
    return df_weight_final

In [3]:
'''
numpy version
'''
def updownlimit(df_nowprice_pct,df_weight):
    '''
    INPUT:
    df_nowprice_pct:为当前时间点的价格相较于昨日收盘价的涨跌幅，如现在为10:30，则为10:30_price - precloseprice 
    df_weight：当日交易的目标权重，如今日想要半仓买入某只股票，则当日的weight为0.5，做空反之
    OUTPUT:
    df_weight_final 考虑了当前时间点由于涨跌停限制买不进去情况下的最终权重，其中不能换仓的资产延续上一个交易日的权重
    '''
    # TODO 写成 numpy
    # TODO 类型检验
    if isinstance(df_nowprice_pct,pd.DataFrame):
#         print 'convert'
        df_nowprice_pct = df_nowprice_pct.values    
    if isinstance(df_weight,pd.DataFrame):
#         print 'convert'
        df_weight = df_weight.values
    if not (df_nowprice_pct.dtype == np.float64 and df_weight.dtype == np.float64):
        raise TypeError("input data must be numpy.float64")
    if df_nowprice_pct.shape != df_weight.shape:
        raise TypeError("input data must has same dims.")
    
    # inf check
    if (np.isinf(df_weight)|np.isneginf(df_weight)).any():
        raise ValueError("input data weight has inf or -inf.")
    if (np.isinf(df_nowprice_pct)|np.isneginf(df_nowprice_pct)).any():
        raise ValueError("input data weight has inf or -inf.")

    
    # small number check
    # large num check
    rownum = df_nowprice_pct.shape[0]
    colnum = df_nowprice_pct.shape[1]

    df_final = np.full(fill_value=0,dtype=np.float64, shape=(rownum,colnum))

    for i in range(rownum):
        if i == 0:
            vec_pct = df_nowprice_pct[0]
            vec_weight_today = df_weight[0]

            vec_pct_bool = np.abs(vec_pct) > 0.097 # 当日涨跌停的股票
            vec_price_isnan_bool = np.isnan(vec_pct) # 停牌的股票
            vec_weight_isnull_bool = np.isnan(vec_weight_today)  # 权重为nan的股票
            
            vec_buy = (~vec_weight_isnull_bool) & (~vec_pct_bool) & (~vec_price_isnan_bool) # TRUE 为确定进行交易的，想买且能买的
            vec_cannotbuy = (vec_pct_bool) | (vec_price_isnan_bool) # True 为 不能交易
            
            out_prev_weight_sum = 1 - np.nansum(np.abs(df_final[0,vec_cannotbuy]))# 抛去不能买卖的，现在还有多少总权重,此时df_final已经被就地修改过了
            
#             df_final[0,~vec_pct_bool] = vec_weight[~vec_pct_bool] # 第一天，直接买入可以买入的
            df_final[0,vec_buy] = (vec_weight_today[vec_buy]/np.nansum(np.abs(vec_weight_today[vec_buy])) * out_prev_weight_sum) #
        else:
            vec_pct = df_nowprice_pct[i]
            vec_weight_prev = df_final[i-1] # 上一个交易日的权重填充
            vec_weight_today = df_weight[i] # 当天的权重

            vec_pct_bool = np.abs(vec_pct) > 0.097 # 涨跌停的股票
            vec_price_isnan_bool = np.isnan(vec_pct) # 停牌的股票
            vec_weight_isnull_bool = np.isnan(vec_weight_today)  # 权重为nan的股票
            # 不想交易的 vec_weight_isnull_bool、涨跌停vec_pct_bool，vec_price_isnan_bool 都视为不进行（不想、不能）买卖的股票
            vec_buy = (~vec_weight_isnull_bool) & (~vec_pct_bool) & (~vec_price_isnan_bool) # TRUE 为确定进行交易的，想买且能买的
            vec_cannotbuy = (vec_pct_bool) | (vec_price_isnan_bool) # True 为 不能交易
            df_final[i,vec_cannotbuy] = vec_weight_prev[vec_cannotbuy] # 凡是今天不能交易的，统一使用上一个交易日填充，df_final第 次修改
            out_prev_weight_sum = 1 - np.nansum(np.abs(df_final[i,:]))# 抛去不能买卖的，现在还有多少总权重,此时df_final已经被就地修改过了
            df_final[i,vec_buy] = (vec_weight_today[vec_buy]/np.nansum(np.abs(vec_weight_today[vec_buy])) * out_prev_weight_sum) # 今天能交易的，延续上一个交易日的权重
    
    return df_final

In [4]:
df_price_pct = pd.DataFrame([[0.01,0.01,0.01]
                             ,[0.05,0.1,0.01]
                             ,[0.01,0.01,0.01]
                             ,[0.01,0.1,-0.099]
                             ,[0,0.1,0.01]
                            ,[np.nan,0.1,0.01]
                            ,[np.nan,0.01,0.01]
                            ,[np.nan,0.01,0.01]
                            ,[0.01,0.01,0.01]
                            ,[0.01,0.01,0.01]
                            ,[0.01,0.0001,0.1]],columns=['A','B','C'])
df_weight = pd.DataFrame([[0.1,0.2,0.7]
                          ,[0.2,-0.1,0.7]
                          ,[0.6,np.nan,0.1]
                          ,[-0.3,0.3,-0.4]
                          ,[0.25,0.5,-0.25]
                         ,[0.5,np.nan,-0.5]
                         ,[0.5,np.nan,-0.5]
                         ,[0.3,0.4,-0.3]
                         ,[0.3,0.4,-0.3]
                         ,[0.3,np.nan,-0.3]
                         ,[0.3,0.4,-0.3]],columns=['A','B','C'])
print df_price_pct
print df_weight

       A       B      C
0   0.01  0.0100  0.010
1   0.05  0.1000  0.010
2   0.01  0.0100  0.010
3   0.01  0.1000 -0.099
4   0.00  0.1000  0.010
5    NaN  0.1000  0.010
6    NaN  0.0100  0.010
7    NaN  0.0100  0.010
8   0.01  0.0100  0.010
9   0.01  0.0100  0.010
10  0.01  0.0001  0.100
       A    B     C
0   0.10  0.2  0.70
1   0.20 -0.1  0.70
2   0.60  NaN  0.10
3  -0.30  0.3 -0.40
4   0.25  0.5 -0.25
5   0.50  NaN -0.50
6   0.50  NaN -0.50
7   0.30  0.4 -0.30
8   0.30  0.4 -0.30
9   0.30  NaN -0.30
10  0.30  0.4 -0.30


In [5]:
'''
array([[ 0.1       ,  0.2       ,  0.7       ],
       [ 0.17777778,  0.2       ,  0.62222222],
       [ 0.85714286,  0.        ,  0.14285714],
       [-0.85714286,  0.        ,  0.14285714],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.28571429, -0.21428571],
       [ 0.3       ,  0.4       , -0.3       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.21428571,  0.28571429, -0.5       ]])
'''

updownlimit(df_price_pct,df_weight)

c:\users\administrator\anaconda2\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in absolute
c:\users\administrator\anaconda2\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in greater


array([[ 0.1       ,  0.2       ,  0.7       ],
       [ 0.17777778,  0.2       ,  0.62222222],
       [ 0.85714286,  0.        ,  0.14285714],
       [-0.85714286,  0.        ,  0.14285714],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.28571429, -0.21428571],
       [ 0.3       ,  0.4       , -0.3       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.21428571,  0.28571429, -0.5       ]])

In [12]:
%tb
import unittest

class TestUplimitDown(unittest.TestCase):
    def setUp(self):
        pass

    def tearDown(self):
        pass

    def test_normal(self):
        df_price_pct = pd.DataFrame(np.array([[ 0.01  ,  0.01  ,  0.01  ],
                                               [ 0.05  ,  0.1   ,  0.01  ],
                                               [ 0.01  ,  0.01  ,  0.01  ],
                                               [ 0.01  ,  0.1   , -0.099 ],
                                               [ 0.    ,  0.1   ,  0.01  ],
                                               [ np.nan,  0.1   ,  0.01  ],
                                               [ np.nan,  0.01  ,  0.01  ],
                                               [ np.nan,  0.01  ,  0.01  ],
                                               [ 0.01  ,  0.01  ,  0.01  ],
                                               [ 0.01  ,  0.01  ,  0.01  ],
                                               [ 0.01  ,  0.0001,  0.1   ]]),columns=['A','B','C'])
        df_weight = pd.DataFrame(np.array([[ 0.1 ,  0.2 ,  0.7 ],
                                           [ 0.2 , -0.1 ,  0.7 ],
                                           [ 0.6 ,np.nan,  0.1 ],
                                           [-0.3 ,  0.3 , -0.4 ],
                                           [ 0.25,  0.5 , -0.25],
                                           [ 0.5 ,np.nan, -0.5 ],
                                           [ 0.5 ,np.nan, -0.5 ],
                                           [ 0.3 ,  0.4 , -0.3 ],
                                           [ 0.3 ,  0.4 , -0.3 ],
                                           [ 0.3 ,np.nan, -0.3 ],
                                           [ 0.3 ,  0.4 , -0.3 ]]),columns=['A','B','C'])
        
        array_final = np.array([[ 0.1       ,  0.2       ,  0.7       ],
                               [ 0.17777778,  0.2       ,  0.62222222],
                               [ 0.85714286,  0.        ,  0.14285714],
                               [-0.85714286,  0.        ,  0.14285714],
                               [ 0.5       ,  0.        , -0.5       ],
                               [ 0.5       ,  0.        , -0.5       ],
                               [ 0.5       ,  0.        , -0.5       ],
                               [ 0.5       ,  0.28571429, -0.21428571],
                               [ 0.3       ,  0.4       , -0.3       ],
                               [ 0.5       ,  0.        , -0.5       ],
                               [ 0.21428571,  0.28571429, -0.5       ]])
        
        df_result = updownlimit(df_price_pct,df_weight)
        self.assertTrue(np.isclose(df_result, array_final).all())
        
        
    def test_inf(self):
        df_price_pct = pd.DataFrame(np.array([[ np.inf  ,  0.01  ,  0.01  ]]),columns=['A','B','C'])
        df_weight = pd.DataFrame(np.array([[ 0.1 ,  0.2 ,  0.7 ]]),columns=['A','B','C'])

        self.assertRaises(ValueError,updownlimit,df_price_pct,df_weight)

        df_price_pct = pd.DataFrame(np.array([[ np.inf  ,  0.01  ,  0.01  ]]),columns=['A','B','C'])
        df_weight = pd.DataFrame(np.array([[ 0.1 ,  np.inf ,  0.7 ]]),columns=['A','B','C'])
#             with self.assertRaises(ValueError):
#                 updownlimit(df_price_pct,df_weight)
        self.assertRaises(ValueError,updownlimit,df_price_pct,df_weight)

    def test_firstday(self):
        df_price_pct = pd.DataFrame(np.array([[ 0.01  ,  0.1  ,  0.01  ]
                                             ,[ 0.01  ,  0.01  ,  0.01  ]]),columns=['A','B','C'])
        df_weight = pd.DataFrame(np.array([[ 0.25 ,  0.25 ,  0.5 ]
                                          ,[ 0.1 ,  0.1 ,  0.8 ]]),columns=['A','B','C'])
        array_final = np.array([[ 0.33333333,  0.        ,  0.66666667],
                                [ 0.1       ,  0.1       ,  0.8       ]])
        result = updownlimit(df_price_pct,df_weight)
        self.assertTrue(np.isclose(result, array_final).all())
        
    def test_type_equal(self):
        df_price_pct1 = pd.DataFrame(np.array([[ 0.01  ,  0.1  ,  0.01  ]
                                             ,[ 0.01  ,  0.01  ,  0.01  ]]),columns=['A','B','C'])
        df_weight1 = pd.DataFrame(np.array([[ 0.25 ,  0.25 ,  0.5 ]
                                          ,[ 0.1 ,  0.1 ,  0.8 ]]),columns=['A','B','C'])
        result1 = updownlimit(df_price_pct1,df_weight1)
        
        df_price_pct2 =  np.array([[ 0.01  ,  0.1  ,  0.01  ]
                                  ,[ 0.01  ,  0.01  ,  0.01  ]]) 
        df_weight2 =  np.array([[ 0.25 ,  0.25 ,  0.5 ]
                               ,[ 0.1 ,  0.1 ,  0.8 ]]) 
        result2 = updownlimit(df_price_pct2,df_weight2)
        
        self.assertTrue(np.isclose(result1, result2).all())
        
    def test_array(self):
        df_price_pct =  np.array([[ 0.01  ,  0.01  ,  0.01  ]
                                  ,[ 0.01  ,  0.01  ,  0.01  ]]) 
        df_weight =  np.array([[ 0.25 ,  0.25 ,  0.5 ]
                               ,[ 0.1 ,  0.1 ,  0.8 ]]) 
        result = updownlimit(df_price_pct,df_weight)
        
        array_final = np.array([[ 0.25      ,  0.25      ,  0.5       ],
                                [ 0.1       ,  0.1       ,  0.8       ]])
        
        self.assertTrue(np.isclose(result, array_final).all())

obj = unittest.main(argv=['ignored', '-v'], exit=False)

No traceback available to show.
test_array (__main__.TestUplimitDown) ... ok
test_firstday (__main__.TestUplimitDown) ... ok
test_inf (__main__.TestUplimitDown) ... ok
test_normal (__main__.TestUplimitDown) ... c:\users\administrator\anaconda2\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in greater
ok
test_type (__main__.TestUplimitDown) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.016s

OK


In [10]:
df_price_pct = pd.DataFrame(np.array([[ 0.01  ,  0.1  ,  0.01  ]
                                             ,[ 0.01  ,  0.01  ,  0.01  ]]),columns=['A','B','C'])
df_weight = pd.DataFrame(np.array([[ 0.25 ,  0.25 ,  0.5 ]
                                  ,[ 0.1 ,  0.1 ,  0.8 ]]),columns=['A','B','C'])
array_final = np.array([[ 0.33333333,  0.        ,  0.66666667],
                    [ 0.1       ,  0.1       ,  0.8       ]])
result = updownlimit(df_price_pct,df_weight)

NameError: name 'array' is not defined

In [11]:
df_price_pct

,A,B,C
0,0.01,0.10,0.01
1,0.01,0.01,0.01


In [12]:
df_weight

,A,B,C
0,0.25,0.25,0.5
1,0.10,0.10,0.8


In [9]:
result

array([[ 0.33333333,  0.        ,  0.66666667],
       [ 0.1       ,  0.1       ,  0.8       ]])

In [43]:
import pdb
df_nowprice_pct = df_price_pct

if not (df_nowprice_pct.values.dtype == np.float64 and df_price_pct.values.dtype == np.float64):
    raise TypeError("input data must be numpy.float64")
if df_nowprice_pct.shape != df_weight.shape:
    raise TypeError("input data must has same dims.")
if isinstance(df_nowprice_pct,pd.DataFrame):
        df_nowprice_pct = df_nowprice_pct.values    
if isinstance(df_weight,pd.DataFrame):
    df_weight = df_weight.values
if (np.isinf(df_weight)|np.isneginf(df_weight)).any():
    raise ValueError("input data weight has inf or -inf.")
if (np.isinf(df_nowprice_pct)|np.isneginf(df_nowprice_pct)).any():
    raise ValueError("input data weight has inf or -inf.")
    
# inf nan check
# small number check
# large num check
rownum = df_nowprice_pct.shape[0]
colnum = df_nowprice_pct.shape[1]

df_final = np.full(fill_value=0,dtype=np.float64, shape=(rownum,colnum))

for i in range(rownum):
    if i == 0:
        vec_pct = df_nowprice_pct[0]
        vec_weight = df_weight[0]
        
        vec_pct_bool = np.abs(vec_pct) > 0.097 # 当日涨跌停的股票
        df_final[0,:] = vec_weight[~vec_pct_bool] # 第一天，直接买入可以买入的
        
    else:
        vec_pct = df_nowprice_pct[i]
        vec_weight_prev = df_final[i-1] # 上一个交易日的权重填充
        vec_weight_today = df_weight[i] # 当天的权重
        
        vec_pct_bool = np.abs(vec_pct) > 0.097 # 涨跌停的股票
        vec_isnan_bool = np.isnan(vec_pct) # 停牌的股票
        vec_isnull_bool = np.isnan(vec_weight_today)  # 权重为nan的股票
        
        vec_canbuy = (~vec_isnull_bool) & (~vec_pct_bool) & (~vec_isnan_bool) # 可继续添加 inf
        
        df_final[i,vec_pct_bool] = vec_weight_prev[vec_pct_bool] # 凡是今天不能交易的，统一使用上一个交易日填充，df_final第 次修改

        out_prev_weight_sum = 1 - np.nansum(np.abs(df_final[i,:]))# 抛去不能买卖的，现在还有多少总权重,此时df_final已经被就地修改过了
        
        
#         if i == 3:
#             pdb.set_trace()

        df_final[i,vec_canbuy] = (vec_weight_today[vec_canbuy]/np.nansum(np.abs(vec_weight_today[vec_canbuy])) * out_prev_weight_sum) # 今天能交易的，延续上一个交易日的权重

c:\users\administrator\anaconda2\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in absolute
c:\users\administrator\anaconda2\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in greater


In [44]:
'''
array([[ 0.1       ,  0.2       ,  0.7       ],
       [ 0.17777778,  0.2       ,  0.62222222],
       [ 0.85714286,  0.        ,  0.14285714],
       [-0.85714286,  0.        ,  0.14285714],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.5       ,  0.        , -0.5       ],
       [ 0.3       ,  0.4       , -0.3       ],
       [ 0.3       ,  0.4       , -0.3       ]])
'''
print df_final


[[ 0.1         0.2         0.7       ]
 [ 0.17777778  0.2         0.62222222]
 [ 0.85714286  0.          0.14285714]
 [-0.85714286  0.          0.14285714]
 [ 0.5         0.         -0.5       ]
 [ 0.          0.         -1.        ]
 [ 0.          0.         -1.        ]
 [ 0.          0.57142857 -0.42857143]
 [ 0.3         0.4        -0.3       ]
 [ 0.5         0.         -0.5       ]
 [ 0.21428571  0.28571429 -0.5       ]]


In [196]:
vec_weight[~vec_pct_bool]/out_prev_weight_sum

array([ 0.22222222,  0.77777778])

In [164]:
df_price_pct

,A,B,C
0,0.01,0.01,0.010
1,0.05,0.10,0.010
2,0.01,0.01,0.010
3,0.01,0.10,-0.099
4,NaN,0.10,0.010
5,NaN,0.10,0.010
6,0.01,0.01,0.010
7,0.01,0.01,0.010


In [173]:
df_weight

,A,B,C
0,0.10,0.2,0.70
1,0.20,-0.1,0.70
2,0.60,NaN,0.10
3,-0.30,0.3,-0.40
4,0.25,0.5,-0.25
5,0.50,NaN,-0.50
6,0.50,NaN,-0.50
7,0.30,0.4,-0.30
8,0.30,0.4,-0.30


In [174]:
updownlimit(df_price_pct,df_weight)

,A,B,C
0,0.100000,0.200000,0.700000
1,0.177778,0.200000,0.622222
2,0.857143,0.000000,0.142857
3,-0.857143,0.000000,0.142857
4,0.500000,0.000000,-0.500000
5,0.500000,0.000000,-0.500000
6,0.500000,0.000000,-0.500000
7,0.500000,0.285714,-0.214286
8,0.300000,0.400000,-0.300000


In [154]:

df_islimit = (~((abs(df_nowprice_pct) > 0.097) | df_nowprice_pct.isnull()) ).astype("float32").replace(0.,np.nan) 
df_isupdown = df_islimit.isnull().astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan
df_isupdown

,A,B,C
0,NaN,NaN,NaN
1,NaN,1.0,NaN
2,NaN,NaN,NaN
3,NaN,1.0,1.0
4,1.0,1.0,NaN
5,1.0,1.0,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN


In [143]:
df_nowprice_pct = df_price_pct 
df_islimit = (~((abs(df_nowprice_pct) > 0.097) | df_nowprice_pct.isnull()) ).astype("float32").replace(0.,np.nan) # 当天涨跌停为nan，其它为1
df_isupdown = df_islimit.isnull().astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan

df_limitbefore_weight = (df_isupdown.shift(-1)*df_weight.fillna(0))            # 涨跌停前一天的权值
df_weight_outlimit = (df_islimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值
df_weight_limitday = ((df_limitbefore_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_limitbefore_weight*df_islimit).fillna(0) # 涨跌停当天的权值,按照前一个交易日的权重进行填充
sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重
df_weight_final_temp = df_weight_outlimit + df_weight_limitday.fillna(0)

# 再来一遍
df_limitbefore_weight = (df_isupdown.shift(-1)*df_weight_final_temp.fillna(0))            # 涨跌停前一天的权值
df_weight_outlimit = (df_islimit*df_weight_final_temp).fillna(0) 
df_weight_limitday = ((df_limitbefore_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_limitbefore_weight*df_islimit).fillna(0) # 涨跌停当天的权值,按照前一个交易日的权重进行填充
sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重
df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)



In [142]:
df_islimit

,A,B,C
0,1.0,1.0,1.0
1,1.0,NaN,1.0
2,1.0,1.0,1.0
3,1.0,NaN,NaN
4,1.0,NaN,1.0
5,1.0,NaN,1.0
6,1.0,1.0,1.0


In [141]:
(df_islimit*df_weight)

,A,B,C
0,0.10,0.2,0.70
1,0.20,NaN,0.70
2,0.60,NaN,0.10
3,-0.30,NaN,NaN
4,0.25,NaN,-0.25
5,0.50,NaN,-0.50
6,0.50,NaN,-0.50
7,NaN,NaN,NaN


In [94]:
df_weight_limitday

,A,B,C
0,NaN,0.0,NaN
1,NaN,0.2,NaN
2,NaN,0.0,0.0
3,NaN,0.0,0.1
4,NaN,0.0,NaN
5,NaN,0.0,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN


In [44]:
df_islimit

,A,B,C
0,1.0,1.0,1.0
1,1.0,NaN,1.0
2,1.0,1.0,1.0
3,1.0,NaN,NaN
4,1.0,NaN,1.0
5,1.0,NaN,1.0
6,1.0,1.0,1.0


In [69]:
df_isupdown

,A,B,C
0,NaN,NaN,NaN
1,NaN,1.0,NaN
2,NaN,NaN,NaN
3,NaN,1.0,1.0
4,NaN,1.0,NaN
5,NaN,1.0,NaN
6,NaN,NaN,NaN


In [68]:
df_weight

,A,B,C
0,0.10,0.2,0.70
1,0.20,-0.1,0.70
2,0.60,NaN,0.10
3,-0.30,0.3,-0.40
4,0.25,0.5,-0.25
5,0.50,NaN,-0.50
6,0.50,NaN,-0.50
7,0.30,0.4,-0.30


In [144]:
df_weight_final

,A,B,C
0,0.100000,0.2,0.700000
1,0.177778,0.2,0.622222
2,0.857143,0.0,0.142857
3,-0.857143,0.0,0.142857
4,0.500000,0.0,-0.500000
5,0.500000,0.0,-0.500000
6,0.500000,0.0,-0.500000
7,0.300000,0.4,-0.300000


In [56]:
df_isupdown_weight*df_islimit

,A,B,C
0,NaN,0.2,NaN
1,NaN,NaN,NaN
2,NaN,0.0,0.1
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


In [58]:
((df_isupdown_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_isupdown_weight*df_islimit).fillna(0)

,A,B,C
0,NaN,0.0,NaN
1,NaN,0.2,NaN
2,NaN,0.0,0.0
3,NaN,0.0,0.1
4,NaN,0.0,NaN
5,NaN,0.0,NaN
6,NaN,NaN,NaN


In [27]:
df_islimit

,A,B,C
0,1.0,1.0,1.0
1,1.0,NaN,1.0
2,1.0,1.0,1.0
3,1.0,NaN,NaN
4,1.0,NaN,1.0
5,1.0,NaN,1.0
6,1.0,1.0,1.0


In [25]:
df_weight_limitday

,A,B,C
0,NaN,0.0,NaN
1,NaN,0.2,NaN
2,NaN,NaN,0.0
3,NaN,NaN,0.1
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


In [23]:
df_isupdown_weight

,A,B,C
0,NaN,0.2,NaN
1,NaN,NaN,NaN
2,NaN,NaN,0.1
3,NaN,0.3,NaN
4,NaN,0.5,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


In [60]:
df_weight_final

,A,B,C
0,0.100000,0.2,0.700000
1,0.177778,0.2,0.622222
2,0.857143,0.0,0.142857
3,-0.900000,0.0,0.100000
4,0.500000,0.0,-0.500000
5,0.500000,0.0,-0.500000
6,0.500000,0.0,-0.500000


In [17]:
df_weight_limitday

,A,B,C
0,NaN,0.0,NaN
1,NaN,0.2,NaN
2,NaN,NaN,0.0
3,NaN,NaN,0.1
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


In [7]:
df = updownlimit(df_price_pct,df_weight)
print abs(df).sum(1)

NameError: name 'updownlimit' is not defined

In [11]:
df_isnotlimit = (~(abs(df_price_pct) > 0.097)).astype("float32").replace(0.,np.nan) # 当天涨跌停为nan，其它为1
print df_isnotlimit

     A    B    C
0  1.0  1.0  1.0
1  1.0  1.0  1.0
2  1.0  NaN  1.0
3  1.0  NaN  1.0


In [12]:
df_islimit = ((abs(df_price_pct) > 0.097)).astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan
print df_islimit

    A    B   C
0 NaN  NaN NaN
1 NaN  NaN NaN
2 NaN  1.0 NaN
3 NaN  1.0 NaN


In [13]:
df_isupdown_weight = df_islimit.shift(-1)*df_weight              # 涨跌停前一天的权值
print df_isupdown_weight

    A    B   C
0 NaN  NaN NaN
1 NaN  0.1 NaN
2 NaN  0.3 NaN
3 NaN  NaN NaN


In [24]:
df_weight_outlimit = (df_isnotlimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值
abs(df_weight_outlimit).sum(1)

print df_weight_outlimit

     A    B    C
0  0.1  0.2  0.7
1  0.2  0.1  0.7
2  0.6  0.0  0.1
3  0.3  0.0  0.4


In [15]:
df_weight_limitday = ((df_isupdown_weight*df_isnotlimit).fillna(1)*df_isnotlimit).ffill().replace(1.,np.nan) - (df_isnotlimit*df_weight).fillna(0)#替换完成，最终的涨跌停当天的权值
print df_weight_limitday 

    A    B   C
0 NaN  NaN NaN
1 NaN  0.0 NaN
2 NaN  0.1 NaN
3 NaN  0.1 NaN


In [19]:
1-abs(df_weight_limitday).sum(1)

0    1.0
1    1.0
2    0.9
3    0.9
dtype: float64

In [25]:
abs(df_weight_outlimit).sum(1)

0    1.0
1    1.0
2    0.7
3    0.7
dtype: float64

In [29]:
# sr_weight_sum_perday = 1-abs(df_weight_limitday).sum(1)
sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)
print df_weight_outlimit

          A    B         C
0  0.100000  0.2  0.700000
1  0.200000  0.1  0.700000
2  0.771429  0.0  0.128571
3  0.385714  0.0  0.514286


In [30]:
df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)
print df_weight_final

          A    B         C
0  0.100000  0.2  0.700000
1  0.200000  0.1  0.700000
2  0.771429  0.1  0.128571
3  0.385714  0.1  0.514286


In [31]:
df_weight_final.sum(1)

0    1.0
1    1.0
2    1.0
3    1.0
dtype: float64

In [51]:
updownlimit(df_price,df_weight)

,A,B,C
0,0.1,0.2,0.7
1,0.2,0.2,0.7
2,0.6,0.2,0.1
3,0.3,0.2,0.4
